In [8]:
import torch
import torch.nn as nn
from torchvision.transforms import transforms
from models.resnet_simclr import ResNetSimCLR
from pyn import Json
from PIL import Image
from tqdm import tqdm
import numpy as np
import os
join = os.path.join

In [9]:
checkpoint = torch.load('./runs/Feb01_12-26-18_pye-MS-7D42/checkpoint_0200.pth.tar')
print(checkpoint.keys())
print(checkpoint['arch'])
model = ResNetSimCLR(checkpoint['arch'], 128)
model.load_state_dict(checkpoint['state_dict'])
model = model.backbone.cuda()
model.fc = nn.Identity()

dict_keys(['epoch', 'arch', 'state_dict', 'optimizer'])
resnet18


In [6]:
PIL2Tesor = transforms.ToTensor()
def img2tensor(img):
    
    img = Image.open(img)
    return PIL2Tesor(img)

In [10]:
permute, label = Json.load('../YJY_Rock/config/BoxImg_224_permute_0.json'),\
                    Json.load('../YJY_Rock/config/labels.json')
tr_set, dev_set = permute['train_data'], permute['test_data']
tr_features, dev_features = [], []
img_root = '../YJY_Rock/'
with torch.no_grad():
    model.eval()
    for img in tqdm(tr_set[:]):
        img = join(img_root, img)
        img_tensor = img2tensor(img).unsqueeze(0).cuda()
        feature = model(img_tensor).cpu()
        torch.cuda.empty_cache()
        tr_features.append(feature)
    for img in tqdm(dev_set[:]):
        img = join(img_root, img)
        img_tensor = img2tensor(img).unsqueeze(0).cuda()
        feature = model(img_tensor).cpu()
        torch.cuda.empty_cache()
        dev_features.append(feature)
tr_features = np.concatenate(tr_features, axis=0)
dev_features = np.concatenate(dev_features, axis=0)
np.save('../YJY_Rock/Features_SimCLR/tr_res18_X.npy', tr_features)
np.save('../YJY_Rock/Features_SimCLR/tt_res18_X.npy', dev_features)

100%|██████████| 12600/12600 [02:20<00:00, 89.88it/s] 


In [11]:
permute, label = Json.load('../YJY_Rock/config/BoxImg_224_permute_0.json'),\
                    Json.load('../YJY_Rock/config/labels.json')
tr_set, dev_set = permute['train_data'], permute['test_data']
tr_label, dev_label = [], []
img_root = '../YJY_Rock/'
with torch.no_grad():
    model.eval()
    for img in tqdm(tr_set[:]):
        tr_label.append(label[img])
    for img in tqdm(dev_set[:]):
        dev_label.append(label[img])
tr_label = np.array(tr_label)
dev_label = np.array(dev_label)
np.save('../YJY_Rock/Features_SimCLR/tr_res18_y.npy', tr_label)
np.save('../YJY_Rock/Features_SimCLR/tt_res18_y.npy', dev_label)

100%|██████████| 12600/12600 [00:00<00:00, 2962842.99it/s]
